In [1]:
import os
import json
import numpy as np
import pandas as pd

this project uses the features extracted by Xirong Li and available at [GitHub][github]

[github]: https://github.com/li-xirong/avs

In [10]:
# define the downloaded directory of MSVD
MSVD_DIR = 'E:\msvd'
BIN_FILE = os.path.join(MSVD_DIR, 'feature.bin')
ID_FILE = os.path.join(MSVD_DIR, 'id.txt')
SHAPE = [41157, 2048]

# the output dir
OUT_DIR = '../../data/resnet_mid_1'

In [11]:
data = np.fromfile(BIN_FILE, dtype=np.float32)
data = data.reshape(SHAPE)

In [12]:
with open(ID_FILE, 'rt') as fid:
    arr_id_with_frame = np.array(fid.read().split())

In [13]:
arr_id = [x[:x.rindex('_')] for x in arr_id_with_frame]
arr_frame = [int(x[x.rindex('_') + 1:]) for x in arr_id_with_frame]
pd_id_frame = pd.DataFrame({'id': arr_id, 'frame': arr_frame})
pd_id_frame.sort_values(['id', 'frame'], inplace=True)

In [ ]:
selected_index = [group.index[group.shape[0] // 2] for _, group in pd_id_frame.groupby('id')]

In [ ]:
for index in selected_index:
    mean_data = data[index]
    np.save(os.path.join(OUT_DIR, arr_id[index] + '.npy'), data[index])

mean of 2 median frames

In [ ]:
selected_index1 = [group.index[group.shape[0] // 2] for _, group in pd_id_frame.groupby('id')]
selected_index2 = [group.index[(group.shape[0] // 2) - 1] for _, group in pd_id_frame.groupby('id')]
selected_index = list(zip(selected_index1, selected_index2))

In [ ]:
# the output dir
OUT_DIR = '../../data/resnet_mid_2'
for index in selected_index:
    mean_data = np.mean(data[list(index)], axis=0)
    np.save(os.path.join(OUT_DIR, arr_id[index[0]] + '.npy'), mean_data)

ResNet by Danny

In [22]:
JSON_FILE = r'E:\feats_vecs_resnet152.json'
# the output dir
OUT_DIR = '../../data/resnet_mean'

In [3]:
a_id = []
a_frame = []
a_feature = []
with open(JSON_FILE, 'rt') as f:
    for line in f:
        d = json.loads(line)
        last_ = d[0].rindex('_')
        a_id.append(d[0][5:last_])
        a_frame.append(int(d[0][last_ + 1:-4]))
        a_feature.append(d[1])

In [18]:
pd_if = pd.DataFrame({'id': a_id, 'frame': a_frame})
pd_if.sort_values(['id', 'frame'], inplace=True)
np_feature = np.array(a_feature, dtype=np.float32)

In [23]:
for vid, group in pd_if.groupby('id'):
    # mean of all frames
    mean_feature = np.mean(np_feature[group.index], axis=0)
    np.save(os.path.join(OUT_DIR, vid + '.npy'), mean_feature)